<a href="https://colab.research.google.com/github/liaorfei/EPIPREP/blob/main/GHSL_POP_dataProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 68.7 MB/s eta 0:00:00


In [6]:
import rasterio as rio
import pandas as pd
import numpy as np

In [7]:
# Estract a dataframe of data from the .tif file that is contained in the .zip
# taken from the GHSL website

# Path to the zip archive and the specific TIFF file within the archive
#dir_path = '/eos/jeodpp/data/projects/ETOHA/DATA/HumanPopData/GlobalHumanSettlementLayer_GHSL'
#csv_name = 'DENGUE_Epico.csv'
#csv_file_path = dir_path + '/' + csv_name
#df = pd.read_csv(csv_file_path)
#print(df.head())

#fold_name = 'GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0'
file_name = 'GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0_R4_C19.tif'

#fold_path = dir_path + '/' + fold_name
tif_file_path = file_name

print(f"Getting data from: {tif_file_path}")

# Open the georeferenced TIFF file directly from the ZIP archive
with rio.open(tif_file_path) as src:
    num_bands = src.count
    #Print the number of bands
    print(f"The TIFF file has {num_bands} band(s).")
    # Read the entire raster as a NumPy array (assuming one band)
    data = src.read(1) #assuming 1 band
    # Get the number of bands from the dataset's count attribute


    # Get the raster's dimensions
    height, width = data.shape

    # Reshape the data to a 1D array where each pixel is a row
    data_reshaped = data.flatten()

    # Create a pandas DataFrame from the reshaped data
    df = pd.DataFrame(data_reshaped, columns=['population'])

    # Get the affine transformation of the raster
    transform = src.transform

    # Generate row and column indices
    row_indices, col_indices = np.indices((height, width))

    # Convert row and column indices to spatial coordinates
    coords = (transform * (col_indices.flatten(), row_indices.flatten()))

    # Add the spatial coordinates to the DataFrame
    df['x'] = coords[0]
    df['y'] = coords[1]

    # If needed, filter out nodata values (e.g., -9999 or another nodata value specified in the raster metadata)
    nodata_value = src.nodata
    if nodata_value is not None:
        df = df[df['population'] != nodata_value]

# Now you have a pandas DataFrame with population values and their spatial coordinates
print(df.head())





Getting data from: GHS_POP_E2030_GLOBE_R2023A_54009_100_V1_0_R4_C19.tif
The TIFF file has 1 band(s).
   population        x          y
0   37.330826 -41000.0  6000000.0
1   80.518326 -40900.0  6000000.0
2   62.383121 -40800.0  6000000.0
3   74.915810 -40700.0  6000000.0
4   58.324982 -40600.0  6000000.0
